<p><font size="5"><b>Projet 2 - Openclassrooms. Data Science </b></font></p>


<p><font size="4"><b>*Analyse des données de systèmes éducatifs </b></font></p>



Source de données : https://datacatalog.worldbank.org/dataset/education-statistics
        
*© 2021, Ilham Noumir  (mailto:ilham.noumir@gmail.com)


## <div class="alert alert-info"> 0. Importation des bibliothèques et chargement des données </div> 

###  Importation des bibliothèques:

In [ ]:

import pandas as pd
import numpy as np
import random 
from sklearn import preprocessing
import matplotlib.pyplot as plt
#import geopandas as gpd
#import folium
#import geoplot
#from wordcloud import WordCloud
#import branca
#from branca.colormap import linear
#import branca.colormap as cm
%matplotlib inline
import seaborn as sns 
import itertools
#from IPython.core.display import display, HTML
#display(HTML("<style>div.output_scroll { height: 250em; }</style>"))
# paramétrage de l'affichage:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_colwidth', -1)
plt.rcParams.update({'font.size':15})

###  Chargement de données: 

###### Le jeu de données des systèmes éducatifs est composé de 5 fichiers CSV

In [ ]:
country= pd.read_csv('EdStatsCountry.csv')
indicateurs= pd.read_csv('EdStatsSeries.csv')
data= pd.read_csv('EdStatsData.csv')
source= pd.read_csv('EdStatsCountry-Series.csv')
description= pd.read_csv('EdStatsFootNote.csv')

## <div class="alert alert-info"> 1. Vérification de la qualité des données: </div> 

###  <p style="color:green;"><b>1.1 Description globale des Dataframes : </p><b>

In [ ]:
list_data= [country, indicateurs, data, source, description]

######## Une fonction pour obtenir les noms des 5 Dataframe ########

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

##### Une fonction pour afficher les dimensions des dataframes #####

def exploration(list): 
    for data in list: 
        print('-'*20)
        print(get_df_name(data))
        print("La dimension de la dataframe {} :" .format(get_df_name(data)) , data.shape)


exploration(list_data)

###  <p style="color:green;"><b>1.2 Données manquantes et données dupliquées sur l'ensemble des Dataframes : </p><b>

In [ ]:
def NanDuplicated(list): 
    for i in list: 
        print('-'*20)
        print(get_df_name(i))
        percentage_NaN= round(i.isnull().sum().sum()/ (len(i) * len(i.columns))*100,2)
        print("La pourcentage totale des valeurs manqauntes dans la dataframe {} :" .format(get_df_name(i)) , percentage_NaN)
        duplicated= sum(i.duplicated())
        print("La somme totale des valeurs dupliquées dans la dataframe {} :" .format(get_df_name(i)) , duplicated)

NanDuplicated(list_data)

###  <p style="color:green;"><b>1.3 Description statistique des données : </p><b> 

In [ ]:
country.describe(include='all')

In [ ]:
indicateurs.describe(include='all')

In [ ]:
data.describe(include='all')

###  <p style="color:green;"><b>1.4 Données manquantes sur les colones et élimination des colones non remplies et inutiles : </p><b> 

###### 1.4.1 Dataframe Country :

In [ ]:
country= pd.read_csv('EdStatsCountry.csv')
country.head(1)

In [ ]:
country.info()

#####  Les valeurs manquantes dans la dataframe country:

In [ ]:
########## La dataframe data_missing représente chaque colone de la dataframe et le pourcentage des NaN correspondant ##########

data_missing= pd.DataFrame((((country.isnull().sum()/len(country))*100 ).sort_values(ascending=False).head(20)))
data_missing= data_missing.reset_index()
data_missing.rename(columns = {0:'% of Total Values'} , inplace= True)
data_missing


In [ ]:
plt.figure(figsize=(20,8))
sns.barplot(y='index', x='% of Total Values', data=data_missing)
plt.title('Nombre de valeurs manquantes par colone', fontsize= 20)
plt.xlabel('Nombre de valeurs manquantes', fontsize=15)
plt.ylabel('colone', fontsize=15)
plt.show()


In [ ]:
#Data_delete regroupe les colones avec un % trés important des NaN et qui ne sont pas d'une grande utilité pour notre étude
data_delete= data_missing.loc[data_missing['% of Total Values']> 48]
data_delete= data_delete.reset_index().rename(columns= {'index':'name_of_colone'})
data_delete

In [ ]:
def delete_nan(missing,data): 
    for row in missing.itertuples():
        name= row.name_of_colone
        data.drop([name], axis='columns', inplace=True)
    print("Les nouvelles dimensions de la dataframe {} sont :".format(get_df_name(data)), data.shape)    
delete_nan(data_delete, country)

## J'ai supprimé également ces trois colones car j'ai jugé qu'ils sont pas nécessaires dans mon étude
country.drop(['Latest household survey','Special Notes','Latest agricultural census'], axis='columns', inplace=True)

###### 1.4.2 .Dataframe indicateurs:

In [ ]:
indicateurs.head(2)

In [ ]:
indicateurs.info()

##### Les valeurs manquantes de la dataframe indicateurs : 

In [ ]:
data_missing2= pd.DataFrame((((indicateurs.isnull().sum()/len(indicateurs))*100 ).sort_values(ascending=False).head(20)))
data_missing2= data_missing2.reset_index()
data_missing2.rename(columns = {0:'% of Total Values'} , inplace= True)
data_missing2

In [ ]:
plt.figure(figsize=(20,8))
sns.barplot(y='index', x='% of Total Values', data=data_missing2)
plt.title('Nombre de valeurs manquantes par colone', fontsize= 20)
plt.xlabel('Nombre de valeurs manquantes', fontsize=15)
plt.ylabel('colone', fontsize=15)
plt.show()

In [ ]:
data_delete2= data_missing2.loc[data_missing2['% of Total Values']> 40]
data_delete2= data_delete2.reset_index().rename(columns= {'index':'name_of_colone'})
data_delete2

In [ ]:
def delete_nan(missing,data): 
    for row in missing.itertuples():
        name= row.name_of_colone
        data.drop([name], axis='columns', inplace=True)
    print("Les nouvelles dimensions de la dataframe {} sont :".format(get_df_name(data)), data.shape) 
delete_nan(data_delete2, indicateurs)

######  1.4.3 . Dataframe data : 

In [ ]:
data.head(3)

In [ ]:
data.info()

In [ ]:
data_missing3= pd.DataFrame((((data.isnull().sum()/len(data))*100 ).sort_values(ascending=False).tail(30)))
data_missing3.rename(columns = {0:'% of Total Values'} , inplace= True)
data_missing3= data_missing3.reset_index()
data_missing3.rename(columns={"index":"name_of_colone"}, inplace=True)
data_missing3

In [ ]:
plt.figure(figsize=(20,8))
#data_year_NaN= data.iloc[:,4:].isnull().sum().sort_values(ascending=True).head(70).reset_index().rename(columns={'index':'colone', 0:'Valeur nulle'})
sns.barplot(x='name_of_colone' , y = '% of Total Values' , data = data_missing3.iloc[:-4,:] , palette="viridis")
plt.title('Nombre des valeurs manqauntes par colone', fontsize= 20)
plt.xlabel('Year', fontsize= 20)
plt.ylabel('% of total values', fontsize= 20)
plt.xticks(fontsize=15)
plt.show()

### ----------------- Bilan de la première partie : Description globale et vérification de la qualité des données ------------- 
1. La Dataframe "Country" est composée de 241 lignes et 32 colones avec un pourcentage totale des NaN qui dépasse 30% et 0 duplicated.
2. La Dataframe "indicateurs" est composée de 3665 lignes et 21 colones avec un pourcentage totale des NaN qui dépasse 71 % et 0 duplicated.
3. La Dataframe "data" est composée de 886930 lignes et 70 colones avec un pourcentage totale des NaN qui dépasse 86 % et 0 duplicated.
4. La Dataframe source comporte 613 lignes et 4 colones  avec un pourcentage totale des NaN qui est égale à 20 % et 0 duplicated.
5. La Dataframe description comporte 643638 lignes et 5 lignes avec un pourcentage totale des NaN qui est égale à 25 % et 0 duplicated.

## <div class="alert alert-info"> 2. Visualisation des données: </div> 

###  <p style="color:green;"><b>2.1 Les pays par région : </p><b>

In [ ]:
f, ax= plt.subplots(figsize=(10,8))
country.groupby('Region')['Short Name'].count().reset_index().sort_values(by='Short Name',ascending= True).plot(kind='barh',x='Region', ax=ax)                                                                                                          
plt.xlabel('Nombre de pays', fontsize=15)
plt.ylabel('Région', fontsize=15)
plt.legend(loc='best', title='Nbr pays par région')
plt.title('Nombre de pays par région', fontsize= 20)
plt.show()

###  <p style="color:green;"><b>2.2 Les pays par classe de revenue : </p><b>

In [ ]:
f,ax= plt.subplots(figsize=(12,8))
country.groupby('Income Group')['Short Name'].count().reset_index().sort_values(by='Short Name', ascending= True).plot(kind='barh',
                                                                                                                       x='Income Group', color='purple',ax=ax)


plt.grid(True)
plt.xlabel('Nombre de pays', fontsize=15)
plt.ylabel('Groupe de revenu', fontsize=15)
plt.legend(loc='best', title='Nbr pays par groupe de revenu')
plt.title('Nombre de pays par classe de revenue', fontsize=20)
plt.show()

###  <p style="color:green;"><b>2.3 Les unités monétaire les plus utilisées par les pays : </p><b>

In [ ]:
f,ax= plt.subplots(figsize=(12,8))
unit= country.groupby('Currency Unit')['Short Name'].count().reset_index().sort_values(by='Short Name', ascending= False).head(10)
unit.plot(kind='bar', x='Currency Unit', color='orange', ax=ax)

plt.grid(True)
plt.xticks(rotation= 60)
plt.xlabel('Unité monétaire', fontsize=15)
plt.ylabel('Nombre de pays par unité monétaire', fontsize=15)
plt.title('Nbr pays par une unité monétaire donnée', fontsize= 20)
plt.legend(loc='best', title='Nbr pays par unité monétaire')
plt.show()

###  <p style="color:green;"><b>2.4 Indicateurs par sujet(Topic) : </p><b>

In [ ]:
Topic=indicateurs.groupby('Topic')['Indicator Name'].count().reset_index().sort_values(by='Indicator Name', ascending=False).head(10)
Topic

In [ ]:
f,ax= plt.subplots(figsize=(12,8))
sns.barplot(x='Topic', y='Indicator Name', data= Topic)
plt.xlabel("Sujet/Topic", fontsize=15)
plt.ylabel('Sujet', fontsize=15)
plt.xticks(rotation=60 , fontsize= 15)
plt.title('Indicateurs par Sujet', fontsize= 20)
plt.legend(loc='best', title='Nbr Indicateurs')
plt.show()

###  <p style="color:green;"><b>2.5 Nuage des mots : </p><b>

In [ ]:
plt.style.use('seaborn')
wrds1 = indicateurs["Indicator Name"].str.split(" ").str[0].value_counts().keys()
wc1 = WordCloud(scale=5,max_words=1000,colormap="rainbow",background_color="grey").generate(" ".join(wrds1))
plt.figure(figsize=(20,14))
plt.imshow(wc1,interpolation="bilinear")
plt.axis("off")
plt.title("Key Words in Indicator Name",color='b' , fontsize=25)
plt.show()

###  <p style="color:green;"><b>2.6 Les préfixes les plus utilisés dans les indiacteurs : </p><b>

In [ ]:
f, ax= plt.subplots(figsize=(18,8))
wrds1 = indicateurs["Indicator Name"].str.split(" ").str[0].value_counts()
indicator_words= pd.DataFrame(wrds1).reset_index().rename(columns={'index':'Indicator','Indicator Name':'Occurence'})

indicator_words['Indicator']= indicator_words['Indicator'].apply(lambda x: x.split(':')[0] if ':' in x else x )
indicator_words= indicator_words.head(20)
indicator_words.plot(kind='bar' , x= 'Indicator', y='Occurence' , ax= ax , color='gray')

plt.xticks(rotation=60, fontsize=15)
plt.yticks(fontsize=15 )
plt.xlabel('Préfixe des indicateurs',fontsize=15)
plt.ylabel("Nbr d'occurence",fontsize=15 )
plt.title('Les préfixes les plus utilisés dans les indicateurs', fontsize=20)
plt.show()

* EGRA : Early Grade Reading Assessment/Manuel pour l'évaluation des compétences fondamentales en lecture 
* Barro-lee: Dataset relatif à l'éducation
* UIS: UNESCO Institut de Statistiques
* SABER
* PASEC


### --------------------------- Bilan de la deuxième partie : Visualisation des données  -------------------------------------
Le but de cette deuxime partie : C'est visuliser les données pour mieux les comprendre

1. Country : contient des informations sur l'économie de chaque pays du monde.
2. Indicateurs: donne des données descriptives sur les indicateurs socio économiques disponibles dans data
3. Data :  donne l'évolution de plusieurs indicateurs pour tous les pays à travers un intervalle de 1970 à 2050

## <div class="alert alert-info"> 3.Filtrage des données: </div> 

###  <p style="color:green;"><b>3.1  L'intervalle des années qu'on peut utilisées dans notre étude : </p><b>

In [ ]:
f, ax= plt.subplots(figsize=(18,8))
data_missing3['taux_de_remplissage']= round((100- data_missing3['% of Total Values'])  ,2)
data_remplissage= data_missing3.iloc[:-4,:]
data_remplissage= data_remplissage.rename(columns= {'name_of_colone':'year'})
sns.barplot(x='year', y= 'taux_de_remplissage', data= data_remplissage)
plt.xlabel('year', fontsize= 15)
plt.ylabel('Taux de remplissage', fontsize=15)
plt.title('Taux de remplissage par année ', fontsize= 20)
plt.xticks(fontsize= 14)
plt.show()

Cette figure nous montre qu'au fur à mesure qu'on s'approche des années 2000, les valeurs manquantes commencent à se diminuer .
Il y a plus de données disponible entre 2000 et 2015. 
Donc l'intervallle d'étude sera de 2000 à 2015

##### Fonction pour supprimer toutes les années qui sont au dessous de 2000 et au dessus de 2015

In [ ]:
data_copy= data.copy()
def select_anne(data): 
    list_annee= []
    for col in data.columns[4:-1] : 
        col= int(col)
        if (col < 2000) | (col > 2015) :
            list_annee.append(col)
    return list_annee

list_annee= select_anne(data_copy)


def delete_annee(list, data):
    for col in data.columns[4:-1] : 
        col= int(col)
        if col in list_annee:
            data.drop([str(col)], axis='columns', inplace=True)
        else : 
            col= col
    return data


data_2000s= delete_annee(list_annee, data_copy)

#J'ai supprimé également les colones Country Code , Indicator Code et Unnamed: 69 car ils apportent pas grande chose à mon étude
data_2000s.drop(columns=['Country Code', 'Indicator Code', 'Unnamed: 69'], inplace=True)

data_2000s.head(5)

###  <p style="color:green;"><b>3.2  Filtrage pays/région : </p><b>

In [ ]:
data['Country Name'].unique()


In [ ]:
data['Country Name'].unique()

No_country=['Arab World', 'East Asia & Pacific',
       'East Asia & Pacific (excluding high income)', 'Euro area',
       'Europe & Central Asia',
       'Europe & Central Asia (excluding high income)', 'European Union',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'Latin America & Caribbean',
       'Latin America & Caribbean (excluding high income)',
       'Least developed countries: UN classification',
       'Low & middle income', 'Low income', 'Lower middle income',
       'Middle East & North Africa',
       'Middle East & North Africa (excluding high income)',
       'Middle income', 'North America', 'OECD members', 'South Asia',
       'Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)',
       'Upper middle income', 'World']

data_2000s= data_2000s[~(data_2000s['Country Name'].isin(No_country))]
data_2000s.head(2)

###  <p style="color:green;"><b>3.3  les pays d'étude : </p><b>

In [ ]:
f , ax= plt.subplots(figsize=(50,300))
## Création d'une colone 'NaN' qui calculent le nombre des valeurs manquantes. 
data_2000s['NaN']= len(data_2000s.columns) - data_2000s.apply(lambda x: x.count(), axis=1)
#Group by qui va permettre par la suite de faire un filtrage sur les pays ou il y a le plus de données.
data_country_nan=data_2000s.groupby('Country Name')['NaN'].sum().reset_index().sort_values(by='NaN', ascending= False)
sns.barplot(x='NaN', y='Country Name', data= data_country_nan)
plt.xlabel('NaN', fontsize= 20)
plt.ylabel('Country Name', fontsize=20)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.title('Remlissage des colones par pays', fontsize=80)

Cette figure nous montre que les petits pays ont un taux de remplissage trés faible. Donc a priori on va se concentrer sur les pays avec une grande population.
Une population supérieure à 5M

###  <p style="color:green;"><b>3.4  Filtrage des indicateurs : </p><b>

#####  Filtrage des indicateurs gendrés

Dans cette étude . je vais choisir mes indicateurs sont se basés sur le gendre('male','female')

In [ ]:
genres= ['male', 'female']
data_2000s= data_2000s[~((data_2000s['Indicator Name'].str.contains('male'))|(data_2000s['Indicator Name'].str.contains('female')))]
data_2000s.head(2)


#####  Filtrage en fonction de la longeur des  noms d'indicateurs: 

Une premiere analyse sur les indicateurs renseignées sur notre Dataset nous montrent qu'il y a des indicateurs avec des longues noms et une faible occurence durant les 10 années de notre étude 
Donc Je vais procéder a un filtrage de mes indicateurs selon leurs longeurs 

In [ ]:
# Année= '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'
data_2000s['longeur_Indicateur']= data_2000s['Indicator Name'].apply(lambda x : len(x))
data_2000s= data_2000s[data_2000s['longeur_Indicateur']<= 100]
data_2000s.drop(columns= {'longeur_Indicateur'}, inplace= True)
data_2000s.head(2)

In [ ]:
data_2000s.shape

###  <p style="color:green;"><b>3.5  Sélection minimale des indicateurs à analyser : </p><b>

In [ ]:
Annee_2000s= []
for col in data_2000s.columns[2:]:
    Annee_2000s.append(col)
data_Indicator_Annee= data_2000s.iloc[:,1:] .groupby('Indicator Name').count().sort_values(by=Annee_2000s, ascending= False)
data_Indicator_Annee= data_Indicator_Annee.reset_index()
data_Indicator_Annee

#####  Heatmap pour  visualiser les le taux de remplissage par Indicateurs et par année :

In [ ]:
plt.figure(figsize=(30,80))
data_heatmap= data_2000s.iloc[:,1:] .groupby('Indicator Name').count().sort_values(by=Annee_2000s, ascending= False)
sns.heatmap(data_heatmap)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.show()

##### Heatmap pour visualiser le taux de remplissage par indicateurs et par pays: 

In [ ]:
data_2000s['NaN']= len(data_2000s.columns) - data_2000s.apply(lambda x: x.count(), axis=1)
data_heatmap = data_2000s[['Country Name', 'Indicator Name', 'NaN']].pivot(index='Country Name', columns='Indicator Name', values='NaN')
data_2000s.drop(columns={'NaN'},inplace=True)
plt.figure(figsize=(30,50))
sns.heatmap(data_heatmap)


### -------------------- Bilan de la troisième partie : Filtrage des données et choix d'indiacteurs  --------------------------------
1. Année d'étude : l'intervalle de 2000 à 2015 qui a un taux de remplissage élevé.
2. pays d'étude : Elimination des pays nouveaux et des petits pays et qui ont un taux de remplissage trés faible 
3. Aprés un croisement des données  selon les années/indicateurs  et puis selon les indicateurs/pays .On a une liste plus synthétique des indicateurs potentiels pour notre étude et qui ont un bon taux de remplissage.
###                                                         ---------------------------------------------------------------------------

Pour un choix plus limité des indicateurs à analyser je me suis basée sur la sémantique des indicateurs(leurs signification métier et leurs importance à évaluer les pays généralement) et aussi sur le taux de remplissage évalué sur la partie en dessus.


L'étude sera portée sur les indicateurs suivants: 
* Population, total
* Internet Users
* GDP per capita (current US)
* Population, ages 15-24, total
* Personal computers (per 100 people) 


## <div class="alert alert-info"> 4. Visualisation des indicateurs choisis: </div> 

###  <p style="color:green;"><b>4.1 PIB/GDP (Produit interne brut) par pays par habitant: </p><b>

In [ ]:
population = data_2000s[data_2000s['Indicator Name'].str.contains('Population, total')]
population= population[population['2015'] < 5000000]
list_pays= population['Country Name'].values
data_2000s= data_2000s[~(data_2000s['Country Name'].isin(list_pays))]
data_pib= data_2000s[data_2000s['Indicator Name'].str.contains('GDP')]
data_pib= data_2000s[data_2000s['Indicator Name']=='GDP per capita (current US$)']
data_pib.head(1)

In [ ]:
#Année= '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'
data_pib_pays=data_pib.groupby(['Country Name','2000', '2005' , '2010', '2015'])['Indicator Name'].max().reset_index()
top_10=data_pib_pays.sort_values(by=['2000', '2005' , '2010', '2015'],ascending= False).head(10)
top_10


In [ ]:
f, ax= plt.subplots(figsize=(20,12))
top_10.plot(kind='bar', x= 'Country Name', ax= ax)
plt.xticks(fontsize= 15, rotation= 80)
plt.yticks(fontsize= 15)
plt.xlabel('Top 10 pays', fontsize= 20)
plt.ylabel('PIB', fontsize= 20)
plt.title('les top 10 pays les mieux classés en pib', fontsize= 25)

plt.show()

#### Courbe de croissance du pib

In [ ]:
f, ax= plt.subplots(figsize=(20,12))
data_pib_pays2=data_pib.groupby(['Country Name','2000', '2002' , '2004', '2006','2008','2010','2012','2014'])['Indicator Name'].max().reset_index()
top_20_=data_pib_pays2.sort_values(by=['2000', '2002' , '2004', '2006','2008','2010','2012','2014'],ascending= False).head(10)
top_20_melt= top_20_.melt(id_vars='Country Name', value_vars=['2000', '2002' , '2004', '2006','2008','2010','2012','2014'], var_name='Year', value_name='value' )
sns.lineplot(x='Year', y='value', data=top_20_melt, hue ='Country Name',  marker="o" , markersize=10  )
plt.xticks(fontsize= 20, rotation= 80)
plt.xlabel('Année de 2000- 2015', fontsize=20)
plt.ylabel('Evolution du PIB',fontsize= 20)
plt.title('Evolution du pib de 2000 à 2015', fontsize= 25)
plt.show()

###  <p style="color:green;"><b>4.2 Population totale : </p><b>

In [ ]:
data_population= data_2000s[data_2000s['Indicator Name']=='Population, total']
data_population['population']= data_population.max(axis=1)
data_population= data_population.sort_values(by='population', ascending=False).head(10)
data_population

In [ ]:
f,ax=plt.subplots(figsize=(20,8))
data_population.reset_index().plot(kind='bar' , x= 'Country Name', y='population', ax=ax)
plt.xlabel('pays' , fontsize=15 )
plt.ylabel('population', fontsize=15)
plt.title('Les pays les plus peuplés au monde', fontsize=20)
plt.legend(loc='best', title=' population par pays')
plt.xticks(fontsize= 20 , rotation= 60)
plt.yticks(fontsize= 20)
plt.show()

###  <p style="color:green;"><b>4.3 Population 15-24 : </p><b>

In [ ]:
data_15_24=data_2000s[data_2000s['Indicator Name']=='Population, ages 15-24, total']
data_15_24['population15/24']= data_15_24.max(axis=1)
data_15_24= data_15_24.sort_values(by='population15/24', ascending= False).head(10)
data_15_24

In [ ]:
f,ax=plt.subplots(figsize=(20,8))
data_15_24.reset_index().plot(kind='bar' , x= 'Country Name', y='population15/24', ax=ax)
plt.xlabel('pays' , fontsize=15 )
plt.ylabel('population15/24', fontsize=25)
plt.title('La population entre 15-24', fontsize=25)
plt.xticks(fontsize= 15)
plt.yticks(fontsize= 15)
plt.legend(loc='best', title=' population 15/24')
plt.show()

###  <p style="color:green;"><b>4.4 Utilisateurs d'Internet : </p><b>

In [ ]:
data_Internet= data_2000s[data_2000s['Indicator Name']=="Internet users (per 100 people)"]
data_Internet.head(2)

In [ ]:
plt.figure(figsize=(20,12))
data_Internet= data_2000s[data_2000s['Indicator Name']=="Internet users (per 100 people)"]
data_Internet['Users']= data_Internet.max(axis=1)
Internet=data_Internet.groupby(['Country Name','2000', '2005' , '2010', '2015'])['Users'].max().reset_index().sort_values(by='Users',ascending=False ).head(10)
usersInetrent= Internet.melt(id_vars= ['Country Name'], value_vars= ['2000', '2005' , '2010', '2015'] , var_name = 'Year', value_name = 'Value' )
usersInetrent['Year']= usersInetrent['Year'].astype('int')
sns.lineplot(x= 'Year', y='Value', hue='Country Name', data= usersInetrent)
plt.legend(loc = 'best' )
plt.title("Evolution des utilisateurs d'internet par pays", size=20)
plt.xticks(fontsize= 15)
plt.yticks(fontsize= 15)
plt.xlabel('Année',fontsize= 20)
plt.ylabel('Utlisateurs Internet', fontsize= 20)
plt.show()

###  <p style="color:green;"><b>4.5 Computers personnels : </p><b>

In [ ]:
plt.figure(figsize=(20,12))
personnalComputer= data_2000s[data_2000s['Indicator Name']=='Personal computers (per 100 people)']
computer= personnalComputer.groupby('Country Name')['2000', '2005' , '2010', '2015'].max(axis=0).reset_index().sort_values(['2000', '2005' , '2010', '2015'], ascending= False).head(10)
Computer= computer.melt(id_vars= ['Country Name'], value_vars= ['2000', '2005' , '2010', '2015'] , var_name = 'Year', value_name = 'Value' )
Computer['Year'] = Computer['Year'].astype('int')
sns.lineplot(x= 'Year', y='Value', hue='Country Name', data= Computer)
plt.legend(loc = 'best' )
plt.title("Evolution de la possession des pc personnel", size=20)
plt.xticks(fontsize= 15)
plt.yticks(fontsize= 15)
plt.xlabel('Année',fontsize= 20)
plt.ylabel('Ordinateurs personnels', fontsize= 20)
plt.show()

###  <p style="color:green;"><b>4.6 Population par région: </p><b>

In [ ]:
countryRegion= country[['Short Name','Region']]

No_country=['Arab World', 'East Asia & Pacific',
       'East Asia & Pacific (excluding high income)', 'Euro area',
       'Europe & Central Asia',
       'Europe & Central Asia (excluding high income)', 'European Union',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'Latin America & Caribbean',
       'Latin America & Caribbean (excluding high income)',
       'Least developed countries: UN classification',
       'Low & middle income', 'Low income', 'Lower middle income',
       'Middle East & North Africa',
       'Middle East & North Africa (excluding high income)',
       'Middle income', 'North America', 'OECD members', 'South Asia',
       'Sub-Saharan Africa', 'Sub-Saharan Africa (excluding high income)',
       'Upper middle income', 'World']

list1= data_2000s['Country Name'].unique()
countryRegion= countryRegion[~(countryRegion['Short Name'].isin(No_country))]

countryRegion= countryRegion[(countryRegion['Short Name'].isin(list1))]

countryRegion.rename(columns={'Short Name':'Country Name'}, inplace=True)

data_2000s = data_2000s.merge(right = countryRegion[['Country Name', 'Region']], 
                  on='Country Name', how='inner')

populationRegion= data_2000s[data_2000s['Indicator Name']=='Population, total']
populationRegion['population']= populationRegion.max(axis=1)
regionPopulation= populationRegion.groupby('Region')['population'].mean().reset_index()
regionPopulation

In [ ]:
plt.figure(figsize=(12,8))
sns.barplot(x='Region', y = 'population', data=regionPopulation)
plt.xticks(rotation=80, fontsize= 15)
plt.xlabel('Région', fontsize= 15)
plt.ylabel('population', fontsize= 15)
plt.title('Population par région', fontsize= 20)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(30,18))
sns.set(style="darkgrid")

populationRegionFinal= populationRegion.sort_values(by='population', ascending=False).groupby('Region').apply(lambda x: x.nlargest(3,'population'))
sns.barplot(x='Region', y = 'population', data=populationRegionFinal , hue='Country Name' )

def change_width(ax, new_value) :
    for patch in ax.patches :
        current_width = patch.get_width()
        diff = current_width - new_value

        # we change the bar width
        patch.set_width(new_value)

        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)

change_width(ax, .6)


plt.xticks(rotation=80, fontsize=30)
plt.yticks(fontsize=15)
plt.xlabel('Région',fontsize= 30)
plt.ylabel('population',fontsize= 30)
plt.legend(loc='best', fontsize =15)
plt.title('population par région avec les top 3 pays pour chaque région', fontsize= 40)
plt.show()


 ###  <p style="color:green;"><b>4.7 Population 15-24 par région: </p><b>

In [ ]:
dataRegion_15_24=data_2000s[data_2000s['Indicator Name']=='Population, ages 15-24, total']
dataRegion_15_24['population15_24']= dataRegion_15_24.max(axis=1)
dataRegion_15_24= dataRegion_15_24.groupby('Region')['population15_24'].mean().reset_index()

dataRegion_15_24= dataRegion_15_24.sort_values(by='population15_24', ascending=False)
regionPopulation= regionPopulation.merge(right=dataRegion_15_24, on='Region', how='inner')

regionPopulation['% population15-24']=(regionPopulation['population15_24']/regionPopulation['population'])*100
regionPopulation


In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
s1 = sns.barplot(x = 'Region', y = 'population', data = regionPopulation, color = 'gray')
s2 = sns.barplot(x = 'Region', y = 'population15_24', data = regionPopulation, color = 'pink')
plt.xticks(rotation=60, fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Région',fontsize= 20)
plt.ylabel('population 15-24/ population totale',fontsize= 20)
plt.legend(loc='best', fontsize =15)
plt.title('Population de 15-24 par rapport à la population totale', fontsize= 25)
plt.show()

 ###  <p style="color:green;"><b>4.8 Utilisateurs d'internet et les computers personnels par région: </p><b>

In [ ]:
fig,ax=plt.subplots(2,1, figsize=(15,15))
fig.suptitle("Evolution des utilisateurs d'Internet et les pc individuelles de 2000 à 2015 " , fontsize=20)

Internet_Region= data_2000s[data_2000s['Indicator Name']=="Internet users (per 100 people)"]
Internet_Region['Users']= Internet_Region.max(axis=1)
internatRegion=Internet_Region.groupby(['Region','2000', '2005' , '2010', '2015'])['Users'].max().reset_index().sort_values(by='Users',ascending=False ).head(20)
internatRegion= internatRegion.melt(id_vars= ['Region'], value_vars= ['2000', '2005' , '2010', '2015'] , var_name = 'Year', value_name = 'Value' )
internatRegion['Year']= internatRegion['Year'].astype('int')
ax[0] =sns.lineplot(x= 'Year', y='Value', hue='Region', data= internatRegion,  ax= ax[0] )
plt.legend(loc = 'best' )
ax[0].set_title("Evolution des utilisateurs d'internet par Région", size=15)
ax[0].set_xlabel('  ')
plt.xticks(fontsize= 15)
plt.yticks(fontsize= 15)

computerRegion= data_2000s[data_2000s['Indicator Name']=='Personal computers (per 100 people)']
regionComuter= computerRegion.groupby('Region')['2000', '2005' , '2010', '2015'].max(axis=0).reset_index().sort_values(['2000', '2005' , '2010', '2015'], ascending= False).head(20)
regionComuter= regionComuter.melt(id_vars= ['Region'], value_vars= ['2000', '2005' , '2010', '2015'] , var_name = 'Year', value_name = 'Value' )
regionComuter['Year'] = regionComuter['Year'].astype('int')
ax[1]= sns.lineplot(x= 'Year', y='Value', hue='Region', data= regionComuter,  ax= ax[1])
plt.legend(loc = 'best' )
ax[1].set_title("Evolution de la possession des pc personnel", size=15)
ax[1].set_xlabel('Année 2000 -2015' , fontsize=15)
plt.xticks(fontsize= 15)
plt.yticks(fontsize= 15)
plt.show()

### -------------------- Bilan de la quatrième partie : Visualisation des indicateurs choisis -------------------------------
Sur cette phase d'étude on a une liste de top10 pays pour chacun des indicateurs choisis


1. les ordinateurs portables : 1.Switezerland 2.United States 3.Sweden 4.Denmark 5.Singapaore 6.Australia 8.Canada 9.Netherlands 10.Finld

2. Le pib :1.Japan 2. Norway 3.Switzeland 4.United States 5.United Arab Emirates 6.Denmark 7.Sweden 8.United Kingdom 9.Netherlands 10.Hong Kong SAR

3. population : 1.Japan  2. Norway 3.Switzeland 4.United States 5.United Arab Emirates 6.Denmark  7.Sweden 8.United Kingdom 9.Netherlands 10.Hong Kong SAR

4. population 15-24 :  1.India 2.China 3.United States 4Indonesia 5. Brazil 6. pakistan 7.Nigeria 8.Bangladesh 9.Russin Federation 10.Ethiopia

5. Utilisateurs d'Internet :1.Switezerland 2.United States 3.Sweden 4.Denmark 5.Singapaore 6.Australia 8.Canada 9.Netherlands 10.Finld

A présent le but sera de choisir 10 pays parmi ces pays choisis pour chacun des indicateurs. 

## <div class="alert alert-info"> 5. Analyse bivariée des indicateurs choisis: </div> 

In [ ]:
# Analyse bivarié
selected_indicator= ['GDP per capita (current US$)','Population, total','Population, ages 15-24, total','Internet users (per 100 people)','Personal computers (per 100 people)']
selected_country= ['Australia', 'Bangladesh', 'Brazil', 'Canada', 'China', 'Denmark',
       'Ethiopia', 'Finland', 'Germany', 'Hong Kong SAR, China', 'India',
       'Indonesia', 'Japan', 'Netherlands', 'Nigeria', 'Norway', 'Pakistan',
       'Russian Federation', 'Singapore', 'Sweden', 'Switzerland',
       'United Arab Emirates', 'United Kingdom', 'United States']


fig, ax = plt.subplots(figsize=(10,6))  

selected_indicator= ['GDP per capita (current US$)','Population, total','Population, ages 15-24, total','Internet users (per 100 people)','Personal computers (per 100 people)']
data_selected=  data_2000s[(data_2000s['Indicator Name'].isin(selected_indicator))&(data_2000s['Country Name'].isin(selected_country))]
data_corr= data_selected.groupby(['Indicator Name']).mean(numeric_only=True)
data_corr = data_corr.transpose()
sns.heatmap(data_corr.corr(),annot = True,square=True)



In [ ]:

selected_indicator= ['GDP per capita (current US$)','Population, total','Population, ages 15-24, total','Internet users (per 100 people)','Personal computers (per 100 people)']
data_selected=  data_2000s[(data_2000s['Indicator Name'].isin(selected_indicator))&(data_2000s['Country Name'].isin(selected_country))]
data2= data_selected.groupby(['Country Name', 'Indicator Name']).mean(numeric_only= True).reset_index()
data3= data2.melt(id_vars= ['Country Name', 'Indicator Name'], value_vars= ['2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015'], var_name='year', value_name='value').dropna()


data4= data3.pivot_table(index=['Country Name','year'] , columns='Indicator Name', values='value').reset_index().dropna()
data_Two= data4[['Country Name', 'year']]
x = data4.iloc[:,2:].values 

### Normalisation des variables: 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data5 = pd.DataFrame(x_scaled ,  columns=['GDP per capita (current US$)',
       'Internet users (per 100 people)',
       'Personal computers (per 100 people)', 'Population, ages 15-24, total',
       'Population, total'] )
dataN= pd.concat([data_Two.reset_index(drop=True), data5.reset_index(drop=True)], axis=1)

dataN= dataN.rename(columns={'GDP per capita (current US$)':'GDP','Internet users (per 100 people)':'Internet users','Personal computers (per 100 people)':'Personal computers','Population, ages 15-24, total':'Population 15-24','Population, total':'Population'})


cols = ['Country Name', 'year', 'GDP', 'Internet users', 'Personal computers',
       'Population 15-24', 'Population']
pp = sns.pairplot(dataN[cols], size=1.8, aspect=1.8,
                  plot_kws=dict(edgecolor="k", linewidth=0.5),
                  diag_kind="kde", diag_kws=dict(shade=True))

fig = pp.fig 
fig.subplots_adjust(top=0.93, wspace=0.5)
fig.show()
t = fig.suptitle('Analyse bivariée des indicateurs choisi', fontsize=18)


In [ ]:

f,ax=plt.subplots(3,1 , figsize=(20,30))

ax[0] =sns.scatterplot(dataN['GDP'], dataN['Population 15-24'], hue=dataN['Country Name'], s=200, ax= ax[0])
ax[0].set_xlabel('GDP', fontsize= 15)
ax[0].set_ylabel('Population 15-24', fontsize= 15)
ax[0].set_title('GDP/Population 15-24', fontsize=20)

ax[1] =sns.scatterplot(dataN['GDP'], dataN['Internet users'], hue=dataN['Country Name'], s=200, ax= ax[1])
ax[1].set_xlabel('GDP', fontsize= 15)
ax[1].set_ylabel('Internet users', fontsize= 15)
ax[1].set_title('GDP/Internet users', fontsize=20)

ax[2] =sns.scatterplot(dataN['Personal computers'], dataN['Internet users'], hue=dataN['Country Name'], s=200, ax= ax[2])
ax[2].set_xlabel('Personal computers', fontsize= 15)
ax[2].set_ylabel('Internet users', fontsize= 15)
ax[2].set_title('Personal computers/Internet users', fontsize=20)
f.show()

### -------------------- Bilan de la cinquième  partie : Analyse bivariée des indicateurs choisis -------------------------------
Sur cette phase en remarque bien que les indicateurs choisis corrèlent entre eux .



## <div class="alert alert-info"> 6. Scoring des pays:  </div> 

 ###  <p style="color:green;"><b>6.1 Listes des pays les mieux classés pour chaque indicateurs : les top 10: </p><b>


In [ ]:
## List PIB :les top 10 des pays 
list_pib= top_10['Country Name']
list_pib= list_pib[:10].values
## List Population :les top 10 des pays 
list_population = data_population['Country Name']
list_population= list_population[:10].values
## List Population15-24 :les top 10 des pays 
list_population15_24= data_15_24['Country Name']
list_population15_24= list_population15_24[:10].values
## List Internet users :les top 10 des pays 
data_Internet= data_2000s[data_2000s['Indicator Name']=="Internet users (per 100 people)"]
data_Internet['Users']= data_Internet.max(axis=1)
Internet=data_Internet.groupby(['Country Name','2000', '2005' , '2010', '2015'])['Users'].max().reset_index().sort_values(by='Users',ascending=False ).head(20)
list_Internet= Internet['Country Name']
list_Internet= list_Internet[:10].values
## List computer values : les top 10 pays
personnalComputer= data_2000s[data_2000s['Indicator Name']=='Personal computers (per 100 people)']
computer= personnalComputer.groupby('Country Name')['2000', '2005' , '2010', '2015'].max(axis=0).reset_index().sort_values(['2000', '2005' , '2010', '2015'], ascending= False).head(20)
list_computer= computer['Country Name']
list_computer= list_computer[:10].values

#### Création d'une liste qui regroupe toutes les listes relatives a chaque pays
list_indicateurs = [list_computer, list_pib, list_population, list_Internet, list_population15_24]
#noms_indicateurs= ['list_computer']
list_titre= str(list_indicateurs)
for i in list_indicateurs : 
    print('-'*20)
    #print(list_titre[i])
    print(i)


 ###  <p style="color:green;"><b>6.2 Attribution des poids pour les pays selon leurs classement pour un indicateur donné . </p><b>

In [ ]:
listPoidsPib= []
listPoidsPopulation15_24= []
listPoidsPopulation= []
listPoidsInternet= []
listPoidsComputer= []

for i in range(len(list_pib),0,-1 ):
    listPoidsPib.append(i)
PoidsPib= list(zip(list_pib, listPoidsPib))
df11= pd.DataFrame(PoidsPib)
df11.rename(columns={0:"pays", 1:"poids_pib"}, inplace=True)
df21= df11.pivot_table(columns='pays')


for i in range(len(list_population15_24),0,-1 ):
    listPoidsPopulation15_24.append(i)
PoidsPopulation15_24= list(zip(list_population15_24, listPoidsPopulation15_24))
df12= pd.DataFrame(PoidsPopulation15_24)
df12.rename(columns={0:"pays", 1:"poids_population15_24"}, inplace=True)
df22= df12.pivot_table(columns='pays')

for i in range(len(list_population),0,-1 ):
    listPoidsPopulation.append(i)
PoidsPopulation= list(zip(list_population, listPoidsPopulation))
df13= pd.DataFrame(PoidsPopulation)
df13.rename(columns={0:"pays", 1:"poids_population"}, inplace=True)
df23= df13.pivot_table(columns='pays')


for i in range(len(list_Internet),0,-1 ):
    listPoidsInternet.append(i)
PoidsInternet= list(zip(list_Internet, listPoidsInternet))
df14= pd.DataFrame(PoidsInternet)
df14.rename(columns={0:"pays", 1:"poids_Internet"}, inplace=True)
df24= df14.pivot_table(columns='pays')


for i in range(len(list_computer),0,-1 ):
    listPoidsComputer.append(i)
PoidsComputer= list(zip(list_computer, listPoidsComputer))
df15= pd.DataFrame(PoidsComputer)
df15.rename(columns={0:"pays", 1:"poids_computer"}, inplace=True)
df25= df15.pivot_table(columns='pays')


df_total= pd.concat([df21, df22, df23, df24,df25], axis=0)
df_total= df_total.fillna(0)
df_total

 ###  <p style="color:green;"><b>6.3 Création de la matrice des pays et du vecteur du poids pour chaque indicateur. </p><b>


In [ ]:
############################## Transformation de la dataframe df_total en matrice ###########################################
matrice_pays= df_total.to_numpy()

###################################### Création du vecteur poids indicateur  #################################################
################################## on va donner des poids différents aux indicateurs à chaque itération ######################
poids_indicateur= []

for i in list(itertools.permutations([1, 2, 3,4,5])):
    array_i= np.array(i)
    poids_indicateur.append(array_i)

######################### matrice finale est le produit du vecteur poids indicateur fois poids pays ######################
matrice_finale=[]
for i in poids_indicateur:
    produit= i.dot(matrice_pays)
    matrice_finale.append(produit)
matrice_finale

In [ ]:
Grid=pd.DataFrame(matrice_finale, columns=['Australia', 'Bangladesh', 'Brazil', 'Canada', 'China', 'Denmark',
       'Ethiopia', 'Finland', 'Germany', 'Hong Kong SAR, China', 'India',
       'Indonesia', 'Japan', 'Netherlands', 'Nigeria', 'Norway', 'Pakistan',
       'Russian Federation', 'Singapore', 'Sweden', 'Switzerland',
       'United Arab Emirates', 'United Kingdom', 'United States']) 
Grid

 ###  <p style="color:green;"><b>6.4 le classement des pays selon leurs score finale. </p><b>


In [ ]:
scores=Grid.mean(axis=0).sort_values(ascending=False).head(10)
scores= pd.DataFrame(scores)
scores=scores.reset_index().rename(columns={'index':'pays', 0: 'scores'})
scores


In [ ]:
plt.figure(figsize=(18,10))
sns.heatmap(Grid, cmap="YlGnBu")
plt.show()

 ###  <p style="color:green;"><b>6.5 Géolocalisation des pays qui ont un meilleur score  pour tous les indicateurs. </p><b>


In [ ]:
geo_json= gpd.read_file('countries.geojson', driver='GeoJSON')
geo_json

In [ ]:
############################# Création d'une liste top 10 pays qui ont les meilleurs scores #################################
countries= ['United States of America', 'Norway', 'Denmark', 'Sweden', 'India', 'China',
       'Switzerland', 'Japan', 'Indonesia', 'Netherlands']


geo_json['ADMIN'].unique()
geo_json.drop(columns='ISO_A3', inplace=True)
geo_json.rename(columns={'ADMIN':'pays'}, inplace=True)

scores.iloc[0,0]='United States of America'
scores['classement']= scores.index+ 1

################### Utilisation de la fonction merge pour joindre le fichier geojson et la dataframe score #################
df_heatmap = pd.merge(geo_json, scores, on='pays')

######################################  Création de l'échelle de la couleur  #################################### 

couleur = linear.YlGnBu_09.scale(min(df_heatmap['scores']), max(df_heatmap['scores']))

style_function = lambda x: {
    'fillColor': couleur(x['properties']['scores']),
    'color': 'black',
    'weight': 1.5,
    'fillOpacity': 0.7
}

############################## Utilisation de la bibliotheque folium pour la création de la map#################################
m = folium.Map(zoom_start=8)
folium.GeoJson(
    df_heatmap,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=['pays', 'scores', 'classement'],
        aliases=['Etat','scores', 'classement'],
        localize=True
    )
).add_to(m)
couleur.add_to(m)
m

 ###  <p style="color:green;"><b>6.6 pour le score maximale pour chacun des pays quel poids pour quel indicateurs à contribuer à ce score maximal??. </p><b>

In [ ]:
colone1= []
matrice_finale2=[]


for i in my_list:
    produit= i.dot(matrice_pays)
    matrice_finale2.append(produit)
    
Grid2=pd.DataFrame(matrice_finale2, columns=['Australia', 'Bangladesh', 'Brazil', 'Canada', 'China', 'Denmark',
       'Ethiopia', 'Finland', 'Germany', 'Hong Kong SAR, China', 'India',
       'Indonesia', 'Japan', 'Netherlands', 'Nigeria', 'Norway', 'Pakistan',
       'Russian Federation', 'Singapore', 'Sweden', 'Switzerland',
       'United Arab Emirates', 'United Kingdom', 'United States']) 

for i in my_list :
    a= i.tolist()
    colone1.append(a)
colone1= pd.DataFrame(colone1)




Grid2=pd.concat([colone1, Grid2], axis=1)
#Grid2.loc[Grid2['United States'].idmax()]



United_States= Grid2[Grid2['United States']==Grid2['United States'].max()].head(1)
Norway= Grid2[Grid2['Norway']==Grid2['Norway'].max()].head(1)
Denmark= Grid2[Grid2['Denmark']==Grid2['Denmark'].max()].head(1)
Sweden = Grid2[Grid2['Sweden']==Grid2['Sweden'].max()].head(1)
China= Grid2[Grid2['China']==Grid2['China'].max()].head(1)
India= Grid2[Grid2['India']==Grid2['India'].max()].head(1)

United_States['pays']= 'United States of America'
Norway['pays']= 'Norway'
Denmark['pays']= 'Denmark'
Sweden['pays']= 'Sweden'
China['pays']= 'China'
India['pays']= 'India'

DataTop6= pd.concat([United_States,Norway,Denmark,Sweden,China,India])
DataTop6= DataTop6.drop(columns= {'Australia','Bangladesh','Brazil','Canada','Ethiopia','Finland','Germany','Hong Kong SAR, China','Indonesia','Japan','Netherlands','Nigeria','Pakistan','Russian Federation','Singapore','Switzerland','United Arab Emirates','United Kingdom'})
DataTop6= DataTop6.rename(columns= {0:'pib',1:'population15-24',2:'population',3:'Internet',4:'Computer'})

DataTop6_heatmap= DataTop6[['pib', 'population15-24', 'population', 'Internet', 'Computer','pays']].set_index('pays')
sns.heatmap(DataTop6_heatmap, annot=True)


### -------------------- Bilan de la sixième partie : Le scoring des pays  -------------------------------
Cette étape nous a permis de choisir 10 pays les mieux scorés :
    1. United States 
    2. Norway
    3. Denmark
    4. Sweden
    5. China
    6. India
    7. Switzerland
    8. Japan
    9. Indonesia
    10. Netherlands
    

    

## <div class="alert alert-info"> 7. Evaluation du potentien des pays:  </div> 

In [ ]:

selected_indicator= ['GDP per capita (current US$)','Population, total','Population, ages 15-24, total','Internet users (per 100 people)','Personal computers (per 100 people)']
data_selected=  data_2000s[(data_2000s['Indicator Name'].isin(selected_indicator))&(data_2000s['Country Name'].isin(selected_country))]
data2= data_selected.groupby(['Country Name', 'Indicator Name']).mean(numeric_only= True).reset_index()
data3= data2.melt(id_vars= ['Country Name', 'Indicator Name'], value_vars= ['2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015'], var_name='year', value_name='value').dropna()

data4= data3.pivot_table(index=['Country Name','year'] , columns='Indicator Name', values='value').reset_index().fillna(0)
data_Two= data4[['Country Name', 'year']]
x = data4.iloc[:,2:].values 

### Normalisation des variables: 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
data5 = pd.DataFrame(x_scaled ,  columns=['GDP per capita (current US$)',
       'Internet users (per 100 people)',
       'Personal computers (per 100 people)', 'Population, ages 15-24, total',
       'Population, total'] )
dataN= pd.concat([data_Two.reset_index(drop=True), data5.reset_index(drop=True)], axis=1)

dataN= dataN.rename(columns={'GDP per capita (current US$)':'GDP','Internet users (per 100 people)':'Internet users','Personal computers (per 100 people)':'Personal computers','Population, ages 15-24, total':'Population 15-24','Population, total':'Population'})

selected_Top10= ['United States', 'Norway', 'Denmark', 'Sweden', 'China','India', 'Switzerland', 'Japan', 'Indonesia', 'Netherlands']
selected_year= ['2010','2011','2012','2013','2014','2015']
dataN= dataN[dataN['year'].isin(selected_year)]
dataN= dataN[dataN['Country Name'].isin(selected_Top10)]
dataN= dataN.sort_values(by='year', ascending=True)
dataN=  dataN.set_index('year')
dataN


In [ ]:
pib_change = dataN.groupby('Country Name')['GDP'].apply(lambda x: x.pct_change().mean()).reset_index(name='taux_croissance_pib').fillna(0)
pib_change['categorie_pib'] = np.where(pib_change['taux_croissance_pib'] < 0, 'decreasing', np.where(pib_change['taux_croissance_pib'] > 0, 'increasing', 'no change'))


population_change = dataN.groupby('Country Name')['Population'].apply(lambda x: x.pct_change().mean()).reset_index(name='taux_croissance_population').fillna(0)
population_change['categorie_population'] = np.where(population_change['taux_croissance_population'] < 0, 'decreasing', np.where(population_change['taux_croissance_population'] > 0, 'increasing', 'no change'))

population15_24_change = dataN.groupby('Country Name')['Population 15-24'].apply(lambda x: x.pct_change().mean()).reset_index(name='taux_croissance_population15_24').fillna(0)
population15_24_change['categorie_population15_24'] = np.where(population15_24_change['taux_croissance_population15_24'] < 0, 'decreasing', np.where(population15_24_change['taux_croissance_population15_24'] > 0, 'increasing', 'no change'))

Internet_users_change = dataN.groupby('Country Name')['Internet users'].apply(lambda x: x.pct_change().mean()).reset_index(name='taux_croissance_Internet').fillna(0)
Internet_users_change['categorie_Internet'] = np.where(Internet_users_change['taux_croissance_Internet'] < 0, 'decreasing', np.where(Internet_users_change['taux_croissance_Internet'] > 0, 'increasing', 'no change'))

Computers_change = dataN.groupby('Country Name')['Personal computers'].apply(lambda x: x.pct_change().mean()).reset_index(name='taux_croissance_Computers').fillna(0)
Computers_change['categorie_computers'] = np.where(Computers_change['taux_croissance_Computers'] < 0, 'decreasing', np.where(Computers_change['taux_croissance_Computers'] > 0, 'increasing', 'no change'))


from functools import reduce
dfs= [pib_change, population_change, population15_24_change,Internet_users_change, Computers_change]
df_final = reduce(lambda left,right: pd.merge(left,right,on='Country Name'), dfs)
df_final= df_final.set_index('Country Name')
df_final



In [ ]:

fig,ax=plt.subplots(2,2 , figsize=(20,15))
fig.suptitle("Taux de croissance des différents indicateurs " , fontsize=20)


ax[0][0] =sns.barplot(x='taux_croissance_pib', y= 'Country Name', data=pib_change , ax=ax[0][0] )
ax[0][0].set_xlabel('taux_croissance_pib', fontsize= 20)
ax[0][0].set_ylabel('Country Name', fontsize= 20)

ax[0][1]=sns.barplot(x='taux_croissance_population', y= 'Country Name', data=population_change, ax= ax[0][1])
ax[0][1].set_xlabel('taux_croissance_population', fontsize= 20)
ax[0][1].set_ylabel('  ', fontsize= 20)

ax[1][0]=sns.barplot(x='taux_croissance_population15_24', y= 'Country Name', data=population15_24_change, ax= ax[1][0])
ax[1][0].set_xlabel('taux_croissance_population15_24', fontsize= 20)
ax[1][0].set_ylabel('Country Name', fontsize= 20)

ax[1][1]=sns.barplot(x='taux_croissance_Internet', y= 'Country Name', data=Internet_users_change, ax= ax[1][1])
ax[1][1].set_xlabel('taux_croissance_Internet', fontsize= 20)
ax[1][1].set_ylabel('  ', fontsize= 20)

fig.show()

In [ ]:
df_final.columns

In [ ]:
df_count = df_final[[ 'categorie_pib', 'categorie_population', 'categorie_population15_24', 'categorie_Internet']]
df_count['Nbr_Increasing'] =df_count.eq('increasing').sum(axis=1).sort_values(ascending=False)
df_count